In [ ]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from gensim.models import Word2Vec 
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
from zipfile import ZipFile
file_name = "Online Retail.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('done')

done


In [ ]:
df = pd.read_excel('/content/Online Retail.xlsx')
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [ ]:
# check for missing values
df.isnull().sum()

# remove missing values
df.dropna(inplace=True)

#convert stock_code to string type since it's a unique combinations of numbers and letters
df['StockCode']= df['StockCode'].astype(str)

In [ ]:
customers = df['CustomerID'].unique().tolist()
random.shuffle(customers)

#customers

In [ ]:
# extract 90% of customer ID's
customers_train = [customers[i] for i in range(round(0.9*len(customers)))]


# split data into train and validation set
train_df = df[df['CustomerID'].isin(customers_train)]
validation_df = df[~df['CustomerID'].isin(customers_train)]


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
purchases_train = []

# populate the list with the product codes
for i in tqdm(customers_train):
    temp = train_df[train_df["CustomerID"] == i]["StockCode"].tolist()
    #print(temp,i)
    purchases_train.append(temp)

100%|██████████| 3935/3935 [00:06<00:00, 631.34it/s]


In [ ]:
purchases_val = []

# populate the list with the product codes
for i in tqdm(validation_df['CustomerID'].unique()):
    temp = validation_df[validation_df["CustomerID"] == i]["StockCode"].tolist()
    purchases_val.append(temp)

100%|██████████| 437/437 [00:00<00:00, 890.93it/s]


In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec

# init callback class
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
            print("")
            print("")
        elif self.epoch % 1 == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
            print("")
            print("")
        
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
model = Word2Vec(window = 10, sg = 1, hs = 0,
                 negative = 10, # for negative sampling
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14)

model.build_vocab(purchases_train, progress_per=200)

model.train(purchases_train, total_examples = model.corpus_count, 
            epochs=10, report_delay=1,compute_loss = True, callbacks=[callback()])

Loss after epoch 0: 3311103.0


Loss after epoch 1: 2970291.0


Loss after epoch 2: 2849733.0


Loss after epoch 3: 2637337.0


Loss after epoch 4: 2503258.0


Loss after epoch 5: 2456641.0


Loss after epoch 6: 2474021.0


Loss after epoch 7: 2261756.0


Loss after epoch 8: 2467268.0


Loss after epoch 9: 2287692.0




(3540586, 3578250)

In [ ]:
model.init_sims(replace=True)
print(model)

Word2Vec(vocab=3146, size=100, alpha=0.03)


In [ ]:
X = model[model.wv.vocab]

X.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


(3146, 100)

In [ ]:
import warnings;
warnings.filterwarnings('ignore')

products = train_df[["StockCode", "Description"]]

# remove duplicates
products.drop_duplicates(inplace=True, subset='StockCode', keep="last")

# create product-ID and product-description dictionary
products_dict = products.groupby('StockCode')['Description'].apply(list).to_dict()

In [ ]:
def similar_products(v, n = 6):
    
    # extract most similar products for the input vector
    ms = model.similar_by_vector(v, topn= n+1)[1:]
    
    # extract name and similarity score of the similar products
    new_ms = []
    for j in ms:
        pair = (products_dict[j[0]][0], j[1])
        new_ms.append(pair)
        
    return new_ms

In [ ]:
similar_products(model['16218']) #"CARTOON PENCIL SHARPENER"

[('POPART WOODEN PENCILS ASST', 0.6819592118263245),
 ('LETTER SHAPE PENCIL SHARPENER', 0.6777251958847046),
 ('TEATIME ROUND PENCIL SHARPENER ', 0.6632087230682373),
 ('HOUSE SHAPE PENCIL SHARPENER', 0.6626463532447815),
 ('PIECE OF CAMO STATIONERY SET', 0.6589818000793457),
 ('TEATIME PUSH DOWN RUBBER', 0.6376833319664001)]

In [ ]:
def aggregate_vectors(products):
    product_vec = []
    for i in products:
        try:
            product_vec.append(model[i])
        except KeyError:
            continue
        
    return np.mean(product_vec, axis=0)

In [ ]:
similar_products(aggregate_vectors(purchases_val[8]))

[('SWALLOW WOODEN CHRISTMAS DECORATION', 0.7002742886543274),
 ('HEART WOODEN CHRISTMAS DECORATION', 0.6955248117446899),
 ('ROCKING HORSE GREEN CHRISTMAS ', 0.6927300691604614),
 ('STAR WOODEN CHRISTMAS DECORATION', 0.6871646642684937),
 ('CHRISTMAS TREE DECORATION WITH BELL', 0.6853213310241699),
 ('CHRISTMAS TREE STAR DECORATION', 0.6851031184196472)]

In [ ]:
similar_products(aggregate_vectors(purchases_val[19]))

[('SPACEBOY GIFT WRAP', 0.6460649967193604),
 ('MINI PAINT SET VINTAGE ', 0.6404285430908203),
 ('CLOTHES PEGS RETROSPOT PACK 24 ', 0.6349523663520813),
 ('WRAP FOLK ART', 0.634928286075592),
 ('DOLLY GIRL LUNCH BOX', 0.6334993839263916),
 ('WRAP ALPHABET DESIGN', 0.6279665231704712)]